Testing delta probability for weeklies and monthlies\n",
    - Does delta match probability of occurrence\n",
    - Check Kelly criterion for long straddles/delta neutral back ratios\n",
        - To calculate expected losses and gains, use options pricing module to interpolate prices at given percentage\n",
        moves and use the normal pdf as the probability weightings\n
Check earnings returns post announcement factors\n",
    - Factors:\n",
        - Number of times beaten earnings (Dummy Variable)\n",
        - Consecutive earnings beats\n",
        - Consecutive earnings upsets\n",
        - 3 Month Trend before earnings\n",
        - YTD Trend\n",
        - Momentum of monthly returns (20 day, 60 day)\n"
        
        
price-to-book (P / B), price-to-earnings (P / E), price-to-free cashflow (P / FCF) and enterprise value-to-EBITDA (EV / EBITDA

In [1]:
import datetime as dt

import numpy as np
import pandas as pd
from pandas_datareader.data import Options
from py_vollib.black_scholes_merton.implied_volatility import *
# ts = TimeSeries(key='5HZEUI5AFJB06BUK',output_format='pandas')
import py_vollib
from py_vollib.black_scholes_merton.implied_volatility import *
from py_vollib.black_scholes_merton.greeks.analytical import *
import plotly
import os
import pandas_market_calendars as mcal
import json
from pandas.io.json import json_normalize
import urllib.request as req
import time
nyse = mcal.get_calendar('NYSE')
from helpers import *

from scipy.stats import norm as norm
from yahoo_query import *

In [2]:
# '''
# functions list:

#     maturities(dt.datetime()) --> [float(front_wgt), float(back_wgt)]

#     optionslam_scrape(str[ticker]) --> dict[earnings]

#     yahoo_table_parse(str[raw_html_table]) --> DataFrame[ticker]

#     yahoo_earnings(dt.datetime()) --> DataFrame[earnings_on_date]

#     fundamentals(str[ticker]) --> DataFrame[stock_fundamentals]

#     get_fundas(list[ticker_lst]) --> DataFrame[stock_fundamentals]

#     historical_data(str[ticker], int[day_number], int[rolling_window], outsize[str]) --> DataFrame[daily_stock_data]

#     current_volatility(list[ticker_lst], int[roll]) --> DataFrame[stock_volatilities]

#     all_options(str[ticker], bool[greeks]) --> DataFrame[options_chains]

#     earnings_condor(str[ticker], int[max_gap], int[dte_thresh], float[|money_thresh| <= 1]) -- DataFrame[condors], DataFrame[puts], DataFrame[calls]

#     write_excel(str[filename], list[str[sheetnames]], list[dataframes]) --> void()

#     curr_stock_data(str[ticker]) --> DataFrame[stock_info]

#     curr_batch_quotes(list_of_string[tickers]) --> DataFrame[stock_info]

#     past_earnings(str[ticker]) --> DataFrame[earnings_info]

#     earnings_history(str[ticker]) --> [DataFrame[earnings_estimate], DataFrame[past_earnings], DataFrame[earnings_estimate_change]]

#     av_data(str[ticker]) --> DataFrame[ticker_open, ticker_close]

#     av_batch(list_of_str[tickers]) --> DataFrame[tickers_closes]

#     check_mkt_corr(int[corr_rolling_window],int[plot_window]) --> DataFrame[rolling_corr]

#     vvix_check() --> DataFrame[VVIX Data]

#     earnings_longs(list_of_str[ticker], float[bid_ask_spread]) --> DataFrame[option_chains]

#     all_options_v2(str[ticker], int[dte_ub], int[dte_lb], float[moneyness]) --> DataFrame[option_chains]

#     yahoo_options_query(str[ticker], int[dte_ub], int[dte_lb]) --> DataFrame[option_chains]

#     greek_calc(DataFrame[option_chain], str[prem_price_use], str[day_format], float[interest_rate], float[dividend_rate])

#     price_sim(DataFrame[options_df], float[price_change], float[vol_change], int[days_change], str[output = 'All'],
#               str[skew = 'flat'], str[day_format = 'trading'], float[interest_rate = 0.0193], float[dividend_rate = 0],
#               float[prem_price_use = 'Mid'])


#     position_sim(DataFrame[position_df], list_of_int[holdings], int[shares],
#                  float[price_change], float[vol_change], int[dte_change], str[output = 'All'],
#                  str[skew = 'flat'], str[prem_price_use = 'Mid'], str[day_format = 'trading'], 
#                  float[interest_rate = 0.0193], float[dividend_rate = 0])

#     yahoo_fundamentals(list_of_str[tickers]) --> DataFrame[fundamentals]

# '''

stock_list = pd.read_csv('optionablestocks.csv')

start_time = time.time()

def pull_data(ticker):
    yahoo_data = yahoo_query(ticker,dt.datetime(2018,1,1))
    yahoo_data.full_info_query()
    earnings_info = yahoo_data.earnings_history.join(yahoo_data.cashFlowStatementQuarter).join(yahoo_data.incomeStatementQuarter.drop(['netIncome','maxAge'],
                                                                                                                                      axis = 1),
                                                                                               rsuffix='_income').join(yahoo_data.balanceSheetQuarter,
                                                                                                                       rsuffix = '_balance')
    earnings_info['earnBeatsBefore'] = 0
    earnings_info['earnMissBefore'] = 0

    for idx, row in earnings_info.iterrows():
        earnings_info.loc[idx,'earnBeatsBefore'] = len(earnings_info[(earnings_info.index <= idx) & (earnings_info.epsDifference > 0)])
        earnings_info.loc[idx,'earnMissBefore'] = len(earnings_info[(earnings_info.index <= idx) & (earnings_info.epsDifference <= 0)])
    earnings_info = earnings_info.shift(1)


    earnings_moves = past_earnings(ticker).sort_index()
    earnings_moves = earnings_moves[(earnings_moves.index > min(yahoo_data.earnings_history.index) - dt.timedelta(days = 92)) &
                                    (earnings_moves.index <= max(yahoo_data.earnings_history.index))].sort_index()


    earnings_df = pd.concat([earnings_info.reset_index(), 
                             earnings_moves.reset_index()], axis = 1)
    earnings_df.columns = ['quarter' if col == 'index' else col for col in earnings_df.columns.tolist()]
    earnings_df['Underlying'] = ticker

    ### separate df for current key measures
    keyMetrics = yahoo_data.profile.join(yahoo_data.keyStats).join(yahoo_data.finData, rsuffix = '_finData')

    return (earnings_df, keyMetrics)

def download_yahoo_data(ticker_list, retries = 10):

    earnings_lst = []
    keyStats_lst = []

    item_counter = 0
    total_length = len(ticker_list)
    failed_list = []

    for ticker in ticker_list:
        try:
            curr_earnings, curr_keyStats = pull_data(ticker)
            earnings_lst.append(curr_earnings)
            keyStats_lst.append(curr_keyStats)
            print('Accepted: {}'.format(ticker))
        except:
            for i in range(retries):
                try:
                    curr_earnings, curr_keyStats = pull_data(ticker)
                    earnings_lst.append(curr_earnings)
                    keyStats_lst.append(curr_keyStats)
                    print('Accepted: {}'.format(ticker))
                except:
                    continue
            print('Failed: {}'.format(ticker))
            failed_list.append(ticker)

        item_counter += 1
        print('{0:.2f}% Completed'.format(item_counter/total_length*100))
        print('{} total failures'.format(len(failed_list)))

    earnings_df = pd.concat(earnings_lst, axis = 0)
    earnings_df = earnings_df.reset_index()[earnings_df.columns]
    keyStats_df = pd.concat(keyStats_lst, axis = 0)

    return earnings_df, keyStats_df, failed_list

def fin_ratios(earnings_df):

    ratios_df = earnings_df[['Underlying','timestamp','quarter','1Year', '1month', '3month', '6month', 
                             'PostEarningsReturn','industry', 'sector']]

In [7]:
# ks = pd.read_csv('long_names.csv', index_col = 0)


# day_number = 300
# roll_window = 20

# returns_lst = []
# for ticker in ks.index.tolist():

#     requesting = True
#     while requesting:
#         try:
#             curr_prices = historical_data(ticker, day_number, roll_window, 'full')[['close']]
#             curr_prices['1year'] = curr_prices.close.pct_change(252)
#             curr_prices['6month'] = curr_prices.close.pct_change(126)
#             curr_prices['3month'] = curr_prices.close.pct_change(63)
#             curr_prices['1month'] = curr_prices.close.pct_change(21)
#             curr_prices = curr_prices.tail(1)
#             curr_prices.index = [ticker]
#             returns_lst.append(curr_prices)

#             requesting = False
#         except:
#             continue


# returns = pd.concat(returns_lst, axis = 0)
# returns.to_csv('returns.csv')

ks = pd.read_csv('keyStats.csv', index_col = 0)

In [81]:
columns = ['currentPrice','52WeekChange', 'SandP52WeekChange', 'beta',
           'bookValue', 'debtToEquity', 'ebitda', 'enterpriseValue',
           'forwardEps', 'forwardPE', 'freeCashflow', 
           'industry', 'overallRisk','priceToBook',
           'quickRatio', 'revenueGrowth', 'earningsGrowth','sector','shortRatio']

new_df = ks[['sector','52WeekChange','pegRatio','priceToBook','revenueGrowth','earningsGrowth']]
new_df['priceToFCF'] = ks['currentPrice']/ks['freeCashflow']
new_df['evToEbitda'] = ks['enterpriseValue']/ks['ebitda']
new_df = new_df.replace('Infinity',np.nan)
new_df['52WeekChange'] = pd.to_numeric(new_df['52WeekChange'])

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [82]:
sector_means = new_df.groupby('sector').mean()
sector_range = new_df.groupby('sector').max() - new_df.groupby('sector').min()
df = new_df.join(sector_means.join(sector_range, rsuffix = '_range'), on = 'sector', lsuffix = '_vals')

In [83]:
df_norm = df[['sector']]
df_norm['52WeekChange'] = (df['52WeekChange_vals'] - df['52WeekChange'])/df['52WeekChange_range']
df_norm['pegRatio'] = (df['pegRatio_vals'] - df['pegRatio'])/df['pegRatio_range']
df_norm['priceToBook'] = (df['priceToBook_vals'] - df['priceToBook'])/df['priceToBook_range']
df_norm['priceToFCF'] = (df['priceToFCF_vals'] - df['priceToFCF'])/df['priceToFCF_range']
df_norm['evToEbitda'] = (df['evToEbitda_vals'] - df['evToEbitda'])/df['evToEbitda_range']
df_norm['revenueGrowth'] = (df['revenueGrowth_vals'] - df['revenueGrowth'])/df['revenueGrowth_range']
df_norm['earningsGrowth'] = (df['earningsGrowth_vals'] - df['earningsGrowth'])/df['earningsGrowth_range']

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexin

In [86]:
df_norm = df_norm.dropna().drop_duplicates()
df_norm['rank'] = df_norm['52WeekChange'] + df_norm['revenueGrowth'] + df_norm['earningsGrowth'] - df_norm[['pegRatio','priceToBook','priceToFCF','evToEbitda']].sum(axis = 1)

In [92]:
df_norm[(df_norm['52WeekChange'] > 0)].sort_values('rank',ascending = False)

sector  52WeekChange  pegRatio  priceToBook  priceToFCF  \
CF           Basic Materials      0.248800 -0.035690     0.000916   -0.016258   
AES                Utilities      0.320342 -0.234601     0.136890   -0.046558   
KR        Consumer Defensive      0.126091  0.038959    -0.044845   -0.059965   
CRM               Technology      0.029708 -0.009695    -0.003018   -0.002698   
HCLP         Basic Materials      0.123934 -0.126576    -0.044328   -0.006497   
FANH      Financial Services      0.654175  0.128156    -0.037254   -0.399227   
EOG                   Energy      0.015561  0.004014     0.163790   -0.020604   
DNR                   Energy      0.782127  0.006250     0.057903    0.010544   
AMZN       Consumer Cyclical      0.205499 -0.005673     0.040381    0.005794   
WWE        Consumer Cyclical      0.693839 -0.001158     0.027831    0.044416   
SHAK       Consumer Cyclical      0.158941 -0.000353    -0.009191   -0.385141   
TROW      Financial Services      0.116075 -0.393983    -0.013349    0.002719   
AAXN             Industrials      0.739605  0.010923     0.007774    0.401855   
GWW              Industrials      0.404292  0.009378     0.015256    0.017155   
DWDP         Basic Materials      0.006219  0.008117    -0.035551   -0.023765   
ZBRA             Industrials      0.185798  0.009373     0.006528    0.013726   
GRUB              Technology      0.244412 -0.009630     0.006040    0.001655   
FE                 Utilities      0.220993 -0.619962     0.065787    0.018906   
SEDG              Technology      0.072596 -0.009949    -0.042944   -0.000142   
SSW              Industrials      0.068030  0.008692    -0.030186   -0.028577   
XPO              Industrials      0.219430  0.008858    -0.016948    0.001707   
VALE         Basic Materials      0.046660 -0.039718    -0.036276   -0.021689   
OLLI      Consumer Defensive      0.417517  0.036525    -0.015279    0.176800   
ANDV                  Energy      0.048434  0.007591     0.019579    0.045815   
ADM       Consumer Defensive      0.031395 -0.010118    -0.065602   -0.026044   
BEAT              Healthcare      0.003652 -0.005658    -0.017236    0.046874   
MOS          Basic Materials      0.190885 -0.016304    -0.045153   -0.017407   
STLD         Basic Materials      0.110489 -0.034683    -0.010278   -0.016861   
IAC               Technology      0.063824 -0.009876    -0.023603   -0.001627   
TTGT              Technology      0.196210 -0.009697    -0.036435    0.004973   
...                      ...           ...       ...          ...         ...   
FFIV              Technology      0.031485 -0.009682     0.003630   -0.001781   
ILMN              Healthcare      0.014599  0.001906     0.009421    0.026437   
AVAV             Industrials      0.216644  0.009881    -0.009728    0.132897   
ADP              Industrials      0.077125  0.009538     0.052575   -0.013834   
ORLY       Consumer Cyclical      0.090173 -0.007678     0.147538    0.024201   
VZ    Communication Services      0.011522  0.057058     0.012645   -0.003569   
MPC                   Energy      0.045094  0.005207     0.038642    0.008371   
INTU              Technology      0.020658 -0.009726     0.155651   -0.002372   
ALGN              Healthcare      0.050336  0.001459     0.038659    0.077011   
PPG          Basic Materials      0.015163  0.000060     0.043640   -0.017189   
UBNT              Technology      0.004619 -0.009809     0.125079   -0.001346   
PX           Basic Materials      0.063831  0.002578     0.086433   -0.016627   
OXY                   Energy      0.001381  0.004611     0.074845   -0.004276   
ECL          Basic Materials      0.036833 -0.000695     0.044896   -0.015197   
MA        Financial Services      0.223010  0.031664     0.413406   -0.013745   
CMG        Consumer Cyclical      0.054811 -0.006036    -0.006253    0.188698   
NEE                Utilities      0.205284 -0.126958     0.079262   -0.042008   
VNOM                  Energy      0.214546  0.009081 